In [1]:
# setup environment
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
import warnings
warnings.filterwarnings('ignore')

In [33]:
# read csv file
df = pd.read_csv('garbage.csv', parse_dates=True, delimiter=';')

# check the data
df.head()

,datum,sammlung,abfallart,abfuhrgebiet,ergebnis,latitude,longitude,wochentag
0,30.09.2021,Kartonsammlung,Fremdstoffe in Sammelware,I,unbe_kannt,47.458607,9.431013,4.0
1,23.09.2022,Unterflurbehälter,Kehrichtsack ohne Gebühr,G,unbe_kannt,47.451193,9.397832,5.0
2,02.10.2020,Unterflurbehälter,Sperrgut,G,unbe_kannt,47.451193,9.397832,5.0
3,26.10.2021,Kartonsammlung,Fremdstoffe in Sammelware,G,unbe_kannt,47.451193,9.397832,2.0
4,04.09.2020,Kehrichtsammlung,Sperrgut,G,unbe_kannt,47.451114,9.397621,5.0


In [34]:
# define grid parameters
lat_sections = 10
long_sections = 20

# find min and max of latitude and longitude
min_lat = df['latitude'].min()
max_lat = df['latitude'].max()
min_long = df['longitude'].min()
max_long = df['longitude'].max()

# create grid
df['lat_bin'] = pd.cut(df['latitude'], bins=pd.interval_range(start=df['latitude'].min(), end=df['latitude'].max(), periods=lat_sections))
df['long_bin'] = pd.cut(df['longitude'], bins=pd.interval_range(start=df['longitude'].min(), end=df['longitude'].max(), periods=long_sections))

# find midpoint of each grid
df['lat_midpoint'] = df['lat_bin'].apply(lambda x: (x.left + x.right) / 2)
df['long_midpoint'] = df['long_bin'].apply(lambda x: (x.left + x.right) / 2)
df['count'] = 1

# remove unnecessary columns and create new dataframe
df2 = df.drop(['latitude', 'sammlung', 'abfallart', 'longitude', 'abfuhrgebiet', 'ergebnis', 'lat_bin', 'long_bin', 'wochentag'], axis=1)

# check the data
df2.head()

# aggregate all data by datums and lat/long midpoint and sum the count
df3 = df2.groupby(['datum', 'lat_midpoint', 'long_midpoint']).sum().reset_index()

# check the data
df3.head()

print(max_lat, min_lat, max_long, min_long)

47.4586072 47.3998588 9.4332366 9.2942733


In [35]:
# Group the data by lat_midpoint and long_midpoint, and sum up the count in each group
grouped_counts = df3.groupby(['lat_midpoint', 'long_midpoint'])['count'].sum().reset_index()

# Filter the grouped data to keep rows with 'count' greater than 0
filtered_combinations = grouped_counts[grouped_counts['count'] > 0]

# Merge the filtered combinations back to the original DataFrame
df4 = df3.merge(filtered_combinations[['lat_midpoint', 'long_midpoint']], on=['lat_midpoint', 'long_midpoint'])

# Check the filtered data
df4.head()

,datum,lat_midpoint,long_midpoint,count
0,01.02.2017,47.402796,9.297747,0
1,01.02.2018,47.402796,9.297747,0
2,01.02.2019,47.402796,9.297747,0
3,01.02.2021,47.402796,9.297747,0
4,01.02.2022,47.402796,9.297747,0


In [36]:
# Create a new column with string concatenation of lat_midpoint and long_midpoint
df4['lat_long_str'] = df4['lat_midpoint'].astype(str) + '_' + df4['long_midpoint'].astype(str)

# Convert the string into a unique integer
df4['grid_id'] = pd.factorize(df4['lat_long_str'])[0]

# Drop the lat_long_str column as it's not needed anymore
df4.drop('lat_long_str', axis=1, inplace=True)

# Check the data
df4.describe()

,count,grid_id
count,122553.000000,122553.000000
mean,0.121270,40.000000
std,0.488006,23.380999
min,0.000000,0.000000
25%,0.000000,20.000000
50%,0.000000,40.000000
75%,0.000000,60.000000
max,13.000000,80.000000


In [37]:
import folium
from folium.plugins import HeatMap

# Calculate the total count for each grid during the entire timeframe
grid_counts = df4.groupby(['lat_midpoint', 'long_midpoint'])['count'].sum().reset_index()

# Create a base map centered on St. Gallen with an appropriate zoom level
map_st_gallen = folium.Map(location=[47.4245, 9.3767], zoom_start=13, max_zoom=15)

# Create a heatmap layer using the grid_counts data
heat_data = [[row['lat_midpoint'], row['long_midpoint'], row['count']] for index, row in grid_counts.iterrows()]
HeatMap(heat_data, radius=30, blur=19).add_to(map_st_gallen)

# Display the heatmap
map_st_gallen

In [38]:
import pandas as pd
import statsmodels.api as sm
from sklearn.metrics import mean_squared_error
import numpy as np
from tqdm import tqdm


# Ensure the 'datum' column is in datetime format
df4['datum'] = pd.to_datetime(df4['datum'], format="%d.%m.%Y")

# Combine the lat_midpoint and long_midpoint columns to create a unique identifier for each grid cell
df4['grid_id'] = df4['lat_midpoint'].astype(str) + "_" + df4['long_midpoint'].astype(str)

# Pivot the data
pivoted_df = df4.pivot_table(index='datum', columns='grid_id', values='count', fill_value=0)

# Resample data to weekly frequency
weekly_df = pivoted_df.resample('W').sum()

# Split the data into train and test sets
train_df = weekly_df.iloc[:-12]
test_df = weekly_df.iloc[-12:]

# Specify the number of weeks to forecast
n_weeks = 3

# Initialize an empty dataframe to store the forecasts
forecasts_df = pd.DataFrame()

# Initialize a list to store the mean squared errors
mse_list = []

# Iterate through all the columns (grid_ids) in the train_df with a progress bar
for grid_id in tqdm(train_df.columns, desc="Training progress", unit="grid cell"):
    # Fit an ARIMA model to the historical data for the current grid cell
    model = sm.tsa.statespace.SARIMAX(train_df[grid_id], order=(1, 0, 0), seasonal_order=(1, 1, 0, 52))
    results = model.fit(maxiter=1000)  # Increase the maximum number of iterations
    
    # Forecast the counts for the next 12 weeks
    forecasts = results.get_forecast(steps=12).predicted_mean
    
    # Calculate the mean squared error
    mse = mean_squared_error(test_df[grid_id], forecasts)
    mse_list.append(mse)
    
    # Forecast the counts for the next n_weeks
    short_term_forecasts = results.get_forecast(steps=n_weeks).predicted_mean
    
    # Add the short-term forecasts to the forecasts_df
    forecasts_df[grid_id] = short_term_forecasts

# Calculate the average mean squared error
average_mse = np.mean(mse_list)
print("Average Mean Squared Error: ", average_mse)

forecasts_df.reset_index(inplace=True)

# Melt the forecasts_df back into the original format
melted_forecasts = forecasts_df.melt(id_vars='index', var_name='grid_id', value_name='count')
melted_forecasts.rename(columns={'index': 'datum'}, inplace=True)

# Split the 'grid_id' column back into 'lat_midpoint' and 'long_midpoint' columns
melted_forecasts[['lat_midpoint', 'long_midpoint']] = melted_forecasts['grid_id'].str.split('_', expand=True).astype(float)

# Reorder columns
melted_forecasts = melted_forecasts[['datum', 'lat_midpoint', 'long_midpoint', 'count']]

Training progress: 100%|██████████| 81/81 [03:07<00:00,  2.32s/grid cell]

Average Mean Squared Error:  1.1116975179877655


In [43]:
# compare the forecasts with the actual data
melted_forecasts['count'].value_counts()

 1.702397e-43    2
 5.170410e-43    2
 3.012916e-42    2
 5.174147e-01    1
 1.248965e+00    1
                ..
 9.139320e-05    1
 6.361650e-29    1
-5.293047e-30    1
 2.704463e-30    1
-3.652535e-26    1
Name: count, Length: 240, dtype: int64

In [39]:
melted_forecasts.head()

,datum,lat_midpoint,long_midpoint,count
0,2022-12-25,47.402796,9.297747,5.174147e-01
1,2023-01-01,47.402796,9.297747,5.174147e-01
2,2023-01-08,47.402796,9.297747,5.174147e-01
3,2022-12-25,47.402796,9.304696,-2.644545e-18
4,2023-01-01,47.402796,9.304696,4.760321e-01


In [40]:
import folium
from folium.plugins import HeatMap

# Calculate the total count for each grid during the entire timeframe
grid_counts = melted_forecasts.groupby(['lat_midpoint', 'long_midpoint'])['count'].sum().reset_index()

# Create a base map centered on St. Gallen with an appropriate zoom level
map_st_gallen = folium.Map(location=[47.4245, 9.3767], zoom_start=13, max_zoom=15)

# Create a heatmap layer using the grid_counts data
heat_data = [[row['lat_midpoint'], row['long_midpoint'], row['count']] for index, row in grid_counts.iterrows()]
HeatMap(heat_data, radius=30, blur=19).add_to(map_st_gallen)

# Display the heatmap
map_st_gallen

In [222]:
import pandas as pd
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import TimeSeriesSplit
from sklearn.metrics import mean_squared_error
from concurrent.futures import ProcessPoolExecutor

def create_lagged_features(data, n_lags):
    for i in range(1, n_lags + 1):
        data[f'lag_{i}'] = data['count'].shift(i)
    return data

def fit_random_forest_and_forecast(grid_id, data, n_weeks):
    data = create_lagged_features(data, n_lags=4).dropna()

    X = data.drop('count', axis=1)
    y = data['count']

    tscv = TimeSeriesSplit(n_splits=5)
    train_indices, test_indices = list(tscv.split(X))[-1]

    X_train, X_test = X.iloc[train_indices], X.iloc[test_indices]
    y_train, y_test = y.iloc[train_indices], y.iloc[test_indices]

    model = RandomForestRegressor(random_state=42)
    model.fit(X_train, y_train)

    mse = mean_squared_error(y_test, model.predict(X_test))

    X_forecast = X.iloc[-n_weeks:]
    forecasts = model.predict(X_forecast)

    return grid_id, forecasts, mse

def parallel_random_forest(weekly_df, n_weeks=4):
    forecasts_df = pd.DataFrame()
    mse_dict = {}
    with ProcessPoolExecutor(max_workers=None) as executor:
        futures = []
        for grid_id in weekly_df.columns:
            futures.append(executor.submit(fit_random_forest_and_forecast, grid_id, weekly_df[[grid_id]], n_weeks))

        for future in futures:
            grid_id, forecasts, mse = future.result()
            forecasts_df[grid_id] = forecasts
            mse_dict[grid_id] = mse

    forecasts_df.reset_index(inplace=True)
    return forecasts_df, mse_dict

# Ensure the 'datum' column is in datetime format
df4['datum'] = pd.to_datetime(df4['datum'], format="%d.%m.%Y")

# Combine the lat_midpoint and long_midpoint columns to create a unique identifier for each grid cell
df4['grid_id'] = df4['lat_midpoint'].astype(str) + "_" + df4['long_midpoint'].astype(str)

# Pivot the data
pivoted_df = df4.pivot_table(index='datum', columns='grid_id', values='count', fill_value=0)

# Resample data to weekly frequency
weekly_df = pivoted_df.resample('W').sum()

# Run the parallel Random Forest model fitting and forecasting
forecasts_df, mse_dict = parallel_random_forest(weekly_df, n_weeks=4)

# Melt the forecasts_df back into the original format
melted_forecasts = forecasts_df.melt(id_vars='index', var_name='grid_id', value_name='count')
melted_forecasts.rename(columns={'index': 'datum'}, inplace=True)

# Split the 'grid_id' column back into 'lat_midpoint' and 'long_midpoint' columns
melted_forecasts[['lat_midpoint', 'long_midpoint']] = melted_forecasts['grid_id'].str.split('_', expand=True).astype(float)

print(melted_forecasts.head())


In [ ]:
import folium
from folium.plugins import HeatMap

# Calculate the total count for each grid during the entire timeframe
grid_counts = melted_forecasts.groupby(['lat_midpoint', 'long_midpoint'])['count'].sum().reset_index()

# Create a base map centered on St. Gallen with an appropriate zoom level
map_st_gallen = folium.Map(location=[47.4245, 9.3767], zoom_start=13, max_zoom=15)

# Create a heatmap layer using the grid_counts data
heat_data = [[row['lat_midpoint'], row['long_midpoint'], row['count']] for index, row in grid_counts.iterrows()]
HeatMap(heat_data, radius=30, blur=19).add_to(map_st_gallen)

# Display the heatmap
map_st_gallen